<a href="https://colab.research.google.com/github/lyogavin/kaggle_experiments/blob/master/seed_finding_3_tweet_sentiment_roberta_with_pp_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
ROOT_PATH="/content/drive/My Drive/kaggle"

In [3]:
!python --version

Python 3.6.9


In [20]:
# Flags:
FOLD_COUNT = 5

NOTEBOOK_ID = 3


DEBUG_LOAD = False
DEBUG_FOLD = False
USE_MULTI_SAMPLE_DROPOUT = True #https://github.com/oleg-yaroshevskiy/quest_qa_labeling/blob/master/step5_model3_roberta_code/model.py#L119
USE_MULTI_SAMPLE_DROPOUT_RATE = 0.5
USE_MULTI_SAMPLE_DROPOUT_SAMPLE = 4

USE_BERT_ALL_LAYERS = False # https://www.kaggle.com/c/google-quest-challenge/discussion/129840
#https://github.com/oleg-yaroshevskiy/quest_qa_labeling/blob/master/step5_model3_roberta_code/model.py#L76

USE_BERT_LAST_N_LAYERS = 4

USE_MULTIPLE_LEARNING_RATE = True #https://github.com/oleg-yaroshevskiy/quest_qa_labeling/blob/master/step5_model3_roberta_code/model.py#L132
USE_MULTIPLE_LEARNING_RATE_TIMES = 3


GEN_PSUEDO_LABELS = False
TRAIN_WITH_PSUEDO_LABELS = False

USE_TRY_MULTI_SEEDING = True

USE_SEQUENCE_BUCKETING = True
USE_SMOOTH_LABELING = True
LABEL_SMOOTH = 0.1


PAD_ID = 1
MAX_LEN = 96

USE_SEED = 42

In [21]:
!pip install tokenizers transformers

In [22]:
import numpy as np
import pandas as pd
import os
import time
import warnings
import random
import torch 
from torch import nn
import torch.optim as optim
from sklearn.model_selection import StratifiedKFold
import tokenizers
from transformers import RobertaModel, RobertaConfig
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

warnings.filterwarnings('ignore')

In [23]:
import re

USE_PP = False

def pp(filtered_output, real_tweet):
    #if not USE_PP:
    #    return filtered_output
    filtered_output = ' '.join(filtered_output.split())
    if len(real_tweet.split()) < 2:
        filtered_output = real_tweet
    else:
        if len(filtered_output.split()) == 1:
            if filtered_output.endswith(".."):
                if real_tweet.startswith(" "):
                    st = real_tweet.find(filtered_output)
                    fl = real_tweet.find("  ")
                    if fl != -1 and fl < st:
                        filtered_output = re.sub(r'(\.)\1{2,}', '', filtered_output)
                    else:
                        filtered_output = re.sub(r'(\.)\1{2,}', '.', filtered_output)
                else:
                    st = real_tweet.find(filtered_output)
                    fl = real_tweet.find("  ")
                    if fl != -1 and fl < st:
                        filtered_output = re.sub(r'(\.)\1{2,}', '.', filtered_output)
                    else:
                        filtered_output = re.sub(r'(\.)\1{2,}', '..', filtered_output)
                return filtered_output
            if filtered_output.endswith('!!'):
                if real_tweet.startswith(" "):
                    st = real_tweet.find(filtered_output)
                    fl = real_tweet.find("  ")
                    if fl != -1 and fl < st:
                        filtered_output = re.sub(r'(\!)\1{2,}', '', filtered_output)
                    else:
                        filtered_output = re.sub(r'(\!)\1{2,}', '!', filtered_output)
                else:
                    st = real_tweet.find(filtered_output)
                    fl = real_tweet.find("  ")
                    if fl != -1 and fl < st:
                        filtered_output = re.sub(r'(\!)\1{2,}', '!', filtered_output)
                    else:
                        filtered_output = re.sub(r'(\!)\1{2,}', '!!', filtered_output)
                return filtered_output

        if real_tweet.startswith(" "):
            filtered_output = filtered_output.strip()
            text_annotetor = ' '.join(real_tweet.split())
            start = text_annotetor.find(filtered_output)
            end = start + len(filtered_output)
            start -= 0
            end += 2
            flag = real_tweet.find("  ")
            if flag < start:
                filtered_output = real_tweet[start:end]

        if "  " in real_tweet and not real_tweet.startswith(" "):
            filtered_output = filtered_output.strip()
            text_annotetor = re.sub(" {2,}", " ", real_tweet)
            start = text_annotetor.find(filtered_output)
            end = start + len(filtered_output)
            start -= 0
            end += 2
            flag = real_tweet.find("  ")
            if flag < start:
                filtered_output = real_tweet[start:end]
    return filtered_output

def prp(filtered_output, real_tweet):
    if not USE_PP:
        return filtered_output
    
    filtered_output = ' '.join(filtered_output.split())
    if len(real_tweet.split()) < 2:
        filtered_output = real_tweet
    else:
        if len(filtered_output.split()) == 1:

            st = real_tweet.find(filtered_output)
            fl = real_tweet.find("  ")
            end = st + len(filtered_output)
            
            #if fl != -1 and fl < st:
            while st != -1 and end < len(real_tweet) and (real_tweet[end] == '.' or real_tweet[end] == '!'):
                filtered_output = filtered_output + real_tweet[end]
                end += 1
                    
                    



        if real_tweet.startswith(" "):
            striped_filtered_output = filtered_output.strip()
            #print(filtered_output)
            text_annotetor = ' '.join(real_tweet.split())
            
            
            start = real_tweet.find(striped_filtered_output)
            end = start + len(striped_filtered_output)
            
        
            
            start -= 0
            end -= 2
            flag = real_tweet.find("  ")
            if flag < start:
                filtered_output0 = text_annotetor[start:end]
                if len(filtered_output0.strip()) != 0:
                    filtered_output = filtered_output0

        if "  " in real_tweet and not real_tweet.startswith(" "):
            striped_filtered_output = filtered_output.strip()
            text_annotetor = re.sub(" {2,}", " ", real_tweet)

            start = real_tweet.find(striped_filtered_output)
            end = start + len(striped_filtered_output)
            
            start -= 0
            end -= 2
            flag = real_tweet.find("  ")
            if flag < start:
                filtered_output0 = text_annotetor[start:end]
                if len(filtered_output0.strip()) != 0:
                    filtered_output = filtered_output0
    return filtered_output

tweet = "  ROFLMAO for the funny web portal  =D"
pred = "funny"
answer = "e funny"
print(pp(pred, tweet))
print(prp(answer, tweet))

tweet = " yea i just got outta one too....i want him back tho  but i feel the same way...i`m cool on dudes for a lil while"
pred = "cool"
answer = "m cool"
print(pp(pred, tweet))
print(prp(answer, tweet))


tweet = "Ow... My shoulder muscle (I can`t remember the name :p) hurts... What did I do?  I don`t even know"
pred = "hurts..."
answer = "hurts.."


print(pp(pred, tweet))
print(prp(answer, tweet))


tweet = " yep... or it should b automatic that if u fall 4 someone that person does 2!or smthng like that... but the way it is sucks!"
pred = "SUCKS!"
answer = "SUCKS!"


print(pp(pred, tweet))
print(prp(answer, tweet))



tweet = " hi holly i`ll volunteer to try it out first for u! hope ur having a fab weekend xoxox..."
pred = "fab"
answer = "g a"


print(pp(pred, tweet))
print(prp(answer, tweet))


e funny
e funny
m cool
m cool
hurts..
hurts..
SUCKS!
SUCKS!
 fab 
g a


In [24]:
"  " in tweet

False

In [25]:
def fifth_prp(select, tweet, sentiment, offsets):
    #print('entering fifth prp')
    ss = tweet.find(select)
    if tweet[max(ss - 2, 0):ss] == '  ':
        ss -= 2
    if ss > 0  and tweet[ss - 1] == ' ':
        ss -= 1

    ee = ss + len(select)

    if re.match(r' [^ ]', tweet) is not None:
        ee -= 1

    ss = max(0, ss)
    if '  ' in tweet[:ss] and sentiment != 'neutral':
        text1 = " ".join(tweet.split())
        sel = text1[ss:ee].strip()
        if len(sel) > 1 and sel[-2] == ' ':
            sel = sel[:-2]

        select = sel

    text1 = " "+" ".join(tweet.split())
    text2 = " ".join(select.split()).lstrip(".,;:")

    idx = text1.find(text2)
    if idx != -1:
        chars = np.zeros((len(text1)))
        chars[idx:idx+len(text2)]=1
        if text1[idx-1]==' ': chars[idx-1] = 1 
    else:
        import pdb;pdb.set_trace()
        chars = np.ones((len(text1)))
        
        
    #tokenizer = tokenizers.ByteLevelBPETokenizer(
    #        vocab_file='../input/roberta-base/vocab.json', 
    #        merges_file='../input/roberta-base/merges.txt', 
    #        lowercase=True,
    #        add_prefix_space=True)
    
    #enc = tokenizer.encode(text1) 

    # ID_OFFSETS
    #offsets = enc.offsets

    # START END TOKENS
    _toks = []

    for i,(a,b) in enumerate(offsets):
        sm = np.mean(chars[a:b])
        #if (sm > 0.6 and chars[a] != 0):  # こうすると若干伸びるけど...
        if (sm > 0.5 and chars[a] != 0): 
            _toks.append(i)
            
    #print("returnning fifth prep")
    return _toks[0], _toks[-1]

    '''
    toks = _toks
    s_tok = sentiment_id[sentiments[k]]
    input_ids[k, :len(enc.ids)+3] = [0, s_tok] + enc.ids + [2]
    attention_mask[k,:len(enc.ids)+3] = 1
    if len(toks)>0:
        start_tokens[k,toks[0]+2] = 1
        end_tokens[k,toks[-1]+2] = 1  
    '''

# Seed

In [26]:
def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True

seed = USE_SEED #42
seed_everything(seed)

# Data Loader

In [27]:
class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, df, max_len=MAX_LEN, use_fifth=True):
        self.df = df
        self.max_len = max_len
        self.labeled = 'selected_text' in df
        self.tokenizer = tokenizers.ByteLevelBPETokenizer(
            vocab_file= ROOT_PATH + '/input/roberta-base/vocab.json', 
            merges_file= ROOT_PATH + '/input/roberta-base/merges.txt', 
            lowercase=True,
            add_prefix_space=True)
        self.use_fifth = use_fifth

    def __getitem__(self, index):
        data = {}
        row = self.df.iloc[index]
        
        #print(f"getting {index}")
        
        ids, masks, tweet, offsets, enc_offsets, padding_len = self.get_input_data(row)
        data['ids'] = ids
        data['original_tweet'] = row.text
        data['masks'] = masks
        data['tweet'] = tweet
        data['offsets'] = offsets
        #data['select'] = row.selected_text
        data['sentiment'] = row.sentiment
        
        if self.labeled:
            try:
                #start_idx, end_idx = self.get_target_idx(row, tweet, offsets)
                if self.use_fifth:
                    start_idx, end_idx = fifth_prp(row.selected_text, row.text, row.sentiment, offsets)
                else:
                    start_idx, end_idx = self.get_target_idx(row, tweet, offsets)
            
            except:
                #print(f"tweet:[{tweet}], selected: [{row.selected_text}] prp: [{prp(row.selected_text, tweet)}]")
                start_idx = 4
                end_idx = len(ids) - 4 - 2
                
            if USE_SEQUENCE_BUCKETING:
              data['start_idx'] = min(start_idx, self.max_len - 1 - padding_len) # start_idx
              data['end_idx'] = min(end_idx, self.max_len - 1 - padding_len) # end_idx
            else:
              data['start_idx'] = start_idx
              data['end_idx'] = end_idx
        
        return data

    def __len__(self):
        return len(self.df)
    
    def get_input_data(self, row):
        tweet = " " + " ".join(row.text.lower().split())
        encoding = self.tokenizer.encode(tweet)
        sentiment_id = self.tokenizer.encode(row.sentiment).ids
        ids = [0] + sentiment_id + [2, 2] + encoding.ids + [2]
        offsets = [(0, 0)] * 4 + encoding.offsets + [(0, 0)]
                
        pad_len = self.max_len - len(ids)
        if pad_len > 0:
            ids += [PAD_ID] * pad_len
            offsets += [(0, 0)] * pad_len
        
        ids = torch.tensor(ids)
        masks = torch.where(ids != PAD_ID, torch.tensor(1), torch.tensor(0))
        offsets = torch.tensor(offsets)
        
        return ids, masks, tweet, offsets, encoding.offsets, pad_len
        
    def get_target_idx(self, row, tweet, offsets):
        
        #pp:
        
        selected_text = prp(row.selected_text.lower(), tweet)
        #selected_text = row.selected_text
        
        if len(selected_text) == 0:
            selected_text = row.selected_text
        
        selected_text = " " +  " ".join(selected_text.lower().split())

        len_st = len(selected_text) - 1
        idx0 = None
        idx1 = None

        for ind in (i for i, e in enumerate(tweet) if e == selected_text[1]):
            if " " + tweet[ind: ind+len_st] == selected_text:
                idx0 = ind
                idx1 = ind + len_st - 1
                break

        char_targets = [0] * len(tweet)
        if idx0 != None and idx1 != None:
            for ct in range(idx0, idx1 + 1):
                char_targets[ct] = 1

        target_idx = []
        for j, (offset1, offset2) in enumerate(offsets):
            if sum(char_targets[offset1: offset2]) > 0:
                target_idx.append(j)

        start_idx = target_idx[0]
        end_idx = target_idx[-1]
        
        return start_idx, end_idx
        
def get_train_val_loaders(df, train_idx, val_idx, batch_size=8):
    train_df = df.iloc[train_idx]
    val_df = df.iloc[val_idx]

    train_loader = torch.utils.data.DataLoader(
        TweetDataset(train_df), 
        batch_size=batch_size, 
        shuffle=True, 
        num_workers=2,
        drop_last=True)

    val_loader = torch.utils.data.DataLoader(
        TweetDataset(val_df), 
        batch_size=batch_size, 
        shuffle=False, 
        num_workers=2)

    dataloaders_dict = {"train": train_loader, "val": val_loader}

    return dataloaders_dict

def get_test_loader(df, batch_size=32):
    loader = torch.utils.data.DataLoader(
        TweetDataset(df), 
        batch_size=batch_size, 
        shuffle=False, 
        num_workers=2)    
    return loader

# Model

In [28]:

class TweetModel(nn.Module):
    def __init__(self):
        super(TweetModel, self).__init__()
        
        config = RobertaConfig.from_pretrained(
            ROOT_PATH + '/input/roberta-base/config.json', output_hidden_states=True)    
        self.roberta = RobertaModel.from_pretrained(
            ROOT_PATH + '/input/roberta-base/pytorch_model.bin', config=config)
        self.dropout = nn.Dropout(0.5)
        self.high_dropout = nn.Dropout(USE_MULTI_SAMPLE_DROPOUT_RATE)
        self.fc = nn.Linear(config.hidden_size, 2)
        nn.init.normal_(self.fc.weight, std=0.02)
        nn.init.normal_(self.fc.bias, 0)


        if USE_BERT_LAST_N_LAYERS == -1:
          n_weights = config.num_hidden_layers
        else:
          n_weights = USE_BERT_LAST_N_LAYERS #config.num_hidden_layers + 1

        self.n_layers = n_weights


        weights_init = torch.zeros(n_weights).float()
        weights_init.data[:-1] = -3
        self.layer_weights = torch.nn.Parameter(weights_init)

        self.multi_layer_dropout = nn.Dropout(0.2)

    def forward(self, input_ids, attention_mask):
        if USE_SEQUENCE_BUCKETING:
          #padding = torch.eq(input_ids, PAD_ID).int()
          padding = torch.where(input_ids == PAD_ID, torch.ones_like(input_ids), torch.zeros_like(input_ids))
          lens = MAX_LEN - torch.sum(padding, -1)
          #max_len = MAX_LEN -1  #torch.max(lens) - 1
          max_len = torch.max(lens)# - 1

          #max_len = torch.clamp(max_len, min=10, max=MAX_LEN)

          #input_ids_last = torch.unsqueeze(input_ids[:, -1], 1)
          #attention_mask_last = torch.unsqueeze(attention_mask[:, -1], 1)
          input_ids = input_ids[:, :max_len]
          attention_mask = attention_mask[:, :max_len]
          #input_ids = torch.cat([input_ids, input_ids_last], dim=1)
          #attention_mask = torch.cat([attention_mask, attention_mask_last], dim=1)
          #tok_ = tok[:, :max_len]

        _, _, hs = self.roberta(input_ids, attention_mask)
         
        if not USE_BERT_ALL_LAYERS:
          x = torch.stack([hs[-1], hs[-2], hs[-3], hs[-4]])
          x = torch.mean(x, 0)
        else:

          #print(f"layers: { [hs[i].shape for i in range(-USE_BERT_LAST_N_LAYERS, 0, 1)]}")



          x = torch.stack(
              [self.multi_layer_dropout(layer[:, :, :]) for layer in [hs[i] for i in range(-self.n_layers, 0, 1)]], dim=3
          )
          x = (torch.softmax(self.layer_weights, dim=0) * x).sum(-1)

        if not USE_MULTI_SAMPLE_DROPOUT:
          x = self.dropout(x)
          x = self.fc(x)
        else:
          # multisample dropout (wut): https://arxiv.org/abs/1905.09788
          x = torch.mean(
              torch.stack(
                  [self.fc(self.high_dropout(x)) for _ in range(USE_MULTI_SAMPLE_DROPOUT_SAMPLE)],
                  dim=0,
              ),
              dim=0,
          )
        start_logits, end_logits = x.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)
                
        return start_logits, end_logits

# Loss Function

In [29]:


# from: https://www.kaggle.com/c/siim-isic-melanoma-classification/discussion/173733
# This loss is similar to CrossEntropyLoss, i.e. it expects logits. Don't use a softmax as your last layer therefore.
class SmoothCrossEntropyLoss(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth_one_hot(targets:torch.Tensor, n_classes:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = torch.empty(size=(targets.size(0), n_classes),
                    device=targets.device) \
                .fill_(smoothing /(n_classes-1)) \
                .scatter_(1, targets.data.unsqueeze(1), 1.-smoothing)
        return targets

    def forward(self, inputs, targets):
        targets = SmoothCrossEntropyLoss._smooth_one_hot(targets, inputs.size(-1),
            self.smoothing)
        lsm = F.log_softmax(inputs, -1)

        if self.weight is not None:
            lsm = lsm * self.weight.unsqueeze(0)

        loss = -(targets * lsm).sum(-1)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

def loss_fn(start_logits, end_logits, start_positions, end_positions):
    if not USE_SMOOTH_LABELING:
      ce_loss = nn.CrossEntropyLoss()
    else:
      ce_loss = SmoothCrossEntropyLoss(smoothing=LABEL_SMOOTH)
    start_loss = ce_loss(start_logits, start_positions)
    end_loss = ce_loss(end_logits, end_positions)    
    total_loss = start_loss + end_loss
    return total_loss


# Evaluation Function

In [30]:
def get_selected_text(text, start_idx, end_idx, offsets):
    selected_text = ""
    for ix in range(start_idx, end_idx + 1):
        selected_text += text[offsets[ix][0]: offsets[ix][1]]
        if (ix + 1) < len(offsets) and offsets[ix][1] < offsets[ix + 1][0]:
            selected_text += " "
    #selected_text = pp(selected_text, text)
    
    return selected_text

def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

def compute_jaccard_score(text, start_idx, end_idx, start_logits, end_logits, offsets):
    start_pred = np.argmax(start_logits)
    end_pred = np.argmax(end_logits)
    if start_pred > end_pred:
        pred = text
    else:
        pred = get_selected_text(text, start_pred, end_pred, offsets)
        
    true = get_selected_text(text, start_idx, end_idx, offsets)
    
    return jaccard(true, pred)

# Training Function

In [31]:
def train_model(model, dataloaders_dict, criterion, optimizer, num_epochs, filename):
    model.cuda()


    jc_ret = 0

    for epoch in range(num_epochs):
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            epoch_loss = 0.0
            epoch_jaccard = 0.0
            
            
            
            for data in (dataloaders_dict[phase]):
                ids = data['ids'].cuda()
                masks = data['masks'].cuda()
                tweet = data['tweet']
                offsets = data['offsets'].numpy()
                start_idx = data['start_idx'].cuda()
                end_idx = data['end_idx'].cuda()
                
                #print((start_idx,end_idx))

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):

                    start_logits, end_logits = model(ids, masks)

                    loss = criterion(start_logits, end_logits, start_idx, end_idx)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    epoch_loss += loss.item() * len(ids)
                    
                    start_idx = start_idx.cpu().detach().numpy()
                    end_idx = end_idx.cpu().detach().numpy()
                    start_logits = torch.softmax(start_logits, dim=1).cpu().detach().numpy()
                    end_logits = torch.softmax(end_logits, dim=1).cpu().detach().numpy()
                    
                    for i in range(len(ids)):                        
                        jaccard_score = compute_jaccard_score(
                            tweet[i],
                            start_idx[i],
                            end_idx[i],
                            start_logits[i], 
                            end_logits[i], 
                            offsets[i])
                        epoch_jaccard += jaccard_score
                    
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_jaccard = epoch_jaccard / len(dataloaders_dict[phase].dataset)
            
            print('Epoch {}/{} | {:^5} | Loss: {:.4f} | Jaccard: {:.4f}'.format(
                epoch + 1, num_epochs, phase, epoch_loss, epoch_jaccard))
            
            if epoch == num_epochs - 1:
              jc_ret = epoch_jaccard
    
    torch.save(model.state_dict(), filename)
    return jc_ret

# Training

In [32]:
num_epochs = 3
batch_size = 32




# **post process:

In [33]:
import re
def modify_punc_length(text, selected_text):
    m = re.search(r'[!\.\?]+$', selected_text)        
    if m is None:
        return selected_text
    
    conti_punc = len(m.group())

    if conti_punc >= 4:
        selected_text = selected_text[:-(conti_punc-2)]
    elif conti_punc == 1:# 元のtextを探しに行く
        tmp = re.sub(r"([\\\*\+\.\?\{\}\(\)\[\]\^\$\|])", r"\\\g<0>", selected_text)
        pat = re.sub(r" ", " +", tmp)
        m = re.search(pat, text)
        f_idx0 = m.start()
        f_idx1 = m.end()

        if f_idx1 != len(text) and text[f_idx1] in ("!", ".", "?"):
            f_idx1 += 1
            selected_text = text[f_idx0:f_idx1]
    return selected_text


import math
def postprocess(tweet, offsets, aa, bb, sentiment):
    text0 = tweet
    text1 = " " + " ".join(tweet.split())
    #enc = tokenizer.encode(text1)

    #aa = np.argmax(start_tokens[k])
    #bb = np.argmax(end_tokens[k])

    ss = offsets[aa][0]
    ee = offsets[bb][1] 
    #st = text1[ss:ee].strip()
    from collections import namedtuple
    Row = namedtuple('Row', ['original_text', 'normalized_text', 'sentiment', 'y_start_char', 'y_end_char'])
    row = Row(
        original_text=text0,
        normalized_text=text1,
        sentiment=sentiment,
        y_start_char=ss,
        y_end_char=ee,
    )
    
    if row.original_text == '':
        return row.normalized_text.strip()
    original_text = row.original_text.replace('\t', '')
    y_start_char = row.y_start_char
    y_end_char = row.y_end_char
    try:
        y_selected_text = row.normalized_text[y_start_char:y_end_char].strip()
    except:
        print(f"err: {row.normalized_text} {(y_start_char,y_end_char)}")
    if (y_end_char < len(row.normalized_text) and row.sentiment != 'neutral' and
        y_selected_text[-1] == '.' and
        (row.normalized_text[y_end_char] == '.' or 
         y_selected_text[-2] == '.')):
        y_selected_text = re.sub('\.+$', '..', y_selected_text)

    tmp = re.sub(r"([\\\*\+\.\?\{\}\(\)\[\]\^\$\|])", r"\\\g<0>", y_selected_text)
    pat = re.sub(r" ", " +", tmp)
    m = re.search(pat, original_text)
    if m is None:
        print(row.normalized_text[y_start_char:y_end_char].strip())
        print(row.normalized_text)
        print(y_selected_text)
    ss2 = m.start()
    ee2 = m.end()
    
    # 'neutral' およびほぼ文書全体が抜き出されるもの
    if row.sentiment == 'neutral' or ((ee2 - ss2) / len(original_text) > 0.75 and  (ee2 - ss2) > 9):
        if len(original_text) > 0 and original_text[0] != '_' and ss2 < 5:
            ss2 = 0 
        if (ee2 < len(original_text)-1 and original_text[ee2:ee2+2] in ('..', '!!', '??', '((', '))')):
            ee2 += 1
        st =  original_text[ss2:ee2].lstrip(' ½¿')
        y_selected_text = st #re.sub(r' .$', '', st)#.strip('`') ###  この一行追加
                
    else:
        if original_text[:int((ss2+ee2) * 0.5) + 1].count('  ') > 0:
            ss = y_start_char
            ee = y_end_char + 1
            if ss > 1 and original_text[ss-1:ss+1] == '..' and  original_text[ss+1] != '.':
                ss -= 1
            st = original_text[ss:ee]#.lstrip(' ½¿')
            y_selected_text = re.sub(r' .$', '', st)#.strip('`') ###  この一行追加
        else:
            if (ee2 < len(original_text)-1 and original_text[ee2:ee2+2] in ('..', '!!', '??', '((', '))')):
                ee2 += 1
            # 先頭の空白分後退
            if  original_text[0] == ' ':
                ss2 -= 1

            y_selected_text = original_text[ss2:ee2].strip(' ½')

            if row.normalized_text[:y_end_char + 5] == " " + row.original_text[:ee2 + 4]: # 簡単のため、長さが同じ場合に限定している
                y_selected_text = modify_punc_length(original_text, y_selected_text)
            
            
    return y_selected_text

In [34]:
!ls -l '/content/drive/My Drive/kaggle/input/roberta-base/pytorch_model.bin'

-rw------- 1 root root 501200538 Aug 17 18:52 '/content/drive/My Drive/kaggle/input/roberta-base/pytorch_model.bin'


In [ ]:
%%time


train_df = pd.read_csv(ROOT_PATH +  '/input/tweet-sentiment-extraction/train.csv')

if DEBUG_LOAD:
    train_df = train_df.head(n=1000)
train_df['text'] = train_df['text'].astype(str)
train_df['selected_text'] = train_df['selected_text'].astype(str)

if os.path.exists(ROOT_PATH +  "/input/seeds_finding/seeds_cv_%d.csv" % NOTEBOOK_ID):
  seed_jaccard = pd.read_csv(ROOT_PATH +  "/input/seeds_finding/seeds_cv_%d.csv" % NOTEBOOK_ID)
  seeds = seed_jaccard['seed'].tolist()
  kfoldseeds = seed_jaccard['kfoldseed'].tolist()
  CVs = seed_jaccard['CV'].tolist()

  MSDs = seed_jaccard['MSD'].tolist()

  SBs = seed_jaccard['SB'].tolist()

  MSDrates = seed_jaccard['MSDrate'].tolist()
  MSDsamples = seed_jaccard['MSDsample'].tolist()
  MLs = seed_jaccard['ML'].tolist()
  MLns = seed_jaccard['MLn'].tolist()
  MLs = seed_jaccard['ML'].tolist()
  MLRs = seed_jaccard['MLR'].tolist()
  MLRtimess = seed_jaccard['MLRtimes'].tolist()
  SLs = seed_jaccard['SL'].tolist()
  SLfactors = seed_jaccard['SLfactor'].tolist()
else:
  seeds = []
  kfoldseeds = []
  CVs = []
  MSDs = []
  SBs = []
  MSDrates = []
  MSDsamples = []
  MLs = []
  MLns = []
  MLs = []
  MLRs = []
  MLRtimess = []
  SLs = []
  SLfactors = []
  

seeding_id = 0

if not USE_TRY_MULTI_SEEDING:
  skf = StratifiedKFold(n_splits=2 if DEBUG_FOLD else FOLD_COUNT, shuffle=True, random_state=seed)


while(True):
  jc_sum = []
  seed = int(time.time())

  seeding_id += 1

  if USE_TRY_MULTI_SEEDING:
    print(f"-----seeding {seeding_id}, seed:{seed}")
    seed_everything(seed)

    skf = StratifiedKFold(n_splits=2 if DEBUG_FOLD else FOLD_COUNT, shuffle=True, random_state=seed)

  for fold, (train_idx, val_idx) in enumerate(skf.split(train_df, train_df.sentiment), start=1): 
      print(f'Fold: {fold}')

      if TRAIN_WITH_PSUEDO_LABELS:
            pl_df= pd.read_csv("psuedo_labels_fold_%d.csv" % (fold - 1))
            if DEBUG_LOAD:
                pl_df = pl_df.head(n=1000)
            pl_df['text'] = pl_df['text'].astype(str)
            pl_df['selected_text'] = pl_df['selected_text'].astype(str)
            #del pl_df['score']
            train_df = train_df.append(pl_df)

      
      if DEBUG_FOLD:
          if fold != 1:
              print(f"DEBUG skip fold:{fold}")
              continue

      model = TweetModel()

      prefix = "roberta"
      def is_backbone(n):
          return prefix in n

      lr=3e-5




      if USE_MULTIPLE_LEARNING_RATE:
        params = list(model.named_parameters())

        optimizer_grouped_parameters = [
            {"params": [p for n, p in params if is_backbone(n)], "lr": lr},
            {"params": [p for n, p in params if not is_backbone(n)], "lr": lr * USE_MULTIPLE_LEARNING_RATE_TIMES},
        ]

        optimizer = torch.optim.AdamW(
          optimizer_grouped_parameters, lr=lr, betas=(0.9, 0.999)
        )
      else:
        optimizer = optim.AdamW(model.parameters(), lr=lr, betas=(0.9, 0.999))

      criterion = loss_fn    
      dataloaders_dict = get_train_val_loaders(train_df, train_idx, val_idx, batch_size)

      jc = train_model(
          model, 
          dataloaders_dict,
          criterion, 
          optimizer, 
          num_epochs,
          f'roberta_fold{fold}.pth')
      
      jc_sum.append(jc)

  print("average jaccard: {:.4f}".format(sum(jc_sum)/len(jc_sum)))
  if USE_TRY_MULTI_SEEDING:
    seeds.append(seed)
    kfoldseeds.append(seed)
    CVs.append(sum(jc_sum)/len(jc_sum))

    MSDs.append(USE_MULTI_SAMPLE_DROPOUT)
    MSDrates.append(USE_MULTI_SAMPLE_DROPOUT_RATE)
    MSDsamples.append(USE_MULTI_SAMPLE_DROPOUT_SAMPLE)
    MLs.append(USE_BERT_ALL_LAYERS)
    MLns.append(USE_BERT_LAST_N_LAYERS)
    MLRs.append(USE_MULTIPLE_LEARNING_RATE)
    MLRtimess.append(USE_MULTIPLE_LEARNING_RATE_TIMES)
    SLs.append(USE_SMOOTH_LABELING)
    SLfactors.append(LABEL_SMOOTH)
    SBs.append(USE_SEQUENCE_BUCKETING)


    seed_jaccard = pd.DataFrame({'seed':seeds,
                                 'CV': CVs, 
                                 'kfoldseed':kfoldseeds,
                                 'MSD':MSDs,
                                 'MSDrate':MSDrates,
                                 'MSDsample':MSDsamples,
                                 'ML':MLs,
                                 'MLn':MLns,
                                 'MLR':MLRs,
                                 'MLRtimes':MLRtimess,
                                 'SL':SLs,
                                 'SLfactor':SLfactors,
                                 'SB':SBs
                                 })

    seed_jaccard.to_csv(ROOT_PATH +  '/input/seeds_finding/seeds_cv_%d.csv' % NOTEBOOK_ID, index=False)
    seed_jaccard.to_csv('seeds_cv_%d.csv' % NOTEBOOK_ID, index=False)

    print(ROOT_PATH +  '/input/seeds_finding/seeds_cv_%d.csv saved' % NOTEBOOK_ID)
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
      print(seed_jaccard)

  if not USE_TRY_MULTI_SEEDING:
    break

-----seeding 1, seed:1598132750
Fold: 1
Epoch 1/3 | train | Loss: 3.1299 | Jaccard: 0.6577
Epoch 1/3 |  val  | Loss: 2.7187 | Jaccard: 0.7151
Epoch 2/3 | train | Loss: 2.6443 | Jaccard: 0.7268
Epoch 2/3 |  val  | Loss: 2.7030 | Jaccard: 0.7208
Epoch 3/3 | train | Loss: 2.5732 | Jaccard: 0.7445
Epoch 3/3 |  val  | Loss: 2.7153 | Jaccard: 0.7248
Fold: 2
Epoch 1/3 | train | Loss: 3.0186 | Jaccard: 0.6769
Epoch 1/3 |  val  | Loss: 2.6839 | Jaccard: 0.7175
Epoch 2/3 | train | Loss: 2.6313 | Jaccard: 0.7307
Epoch 2/3 |  val  | Loss: 2.6346 | Jaccard: 0.7329
Epoch 3/3 | train | Loss: 2.5166 | Jaccard: 0.7514
Epoch 3/3 |  val  | Loss: 2.6630 | Jaccard: 0.7210
Fold: 3
Epoch 1/3 | train | Loss: 3.0233 | Jaccard: 0.6789
Epoch 1/3 |  val  | Loss: 2.7113 | Jaccard: 0.7250
Epoch 2/3 | train | Loss: 2.6375 | Jaccard: 0.7271
Epoch 2/3 |  val  | Loss: 2.6734 | Jaccard: 0.7251
Epoch 3/3 | train | Loss: 2.5201 | Jaccard: 0.7515
Epoch 3/3 |  val  | Loss: 2.6936 | Jaccard: 0.7251
Fold: 4
Epoch 1/3 | train 

In [ ]:
EXPLORE = False

if EXPLORE:

    train_df_explore = pd.read_csv(ROOT_PATH +  '/input/tweet-sentiment-extraction/train.csv')

    train_df_explore.head()
    
    train_df_explore['text'] = train_df_explore['text'].astype(str)
    train_df_explore['selected_text'] = train_df_explore['selected_text'].astype(str)
    
    ds = TweetDataset(train_df_explore)
    
    
    ds1 = TweetDataset(train_df_explore, use_fifth=False)
    
    same_count = 0
    diff_count = 0
    
    for d1, d2 in zip(ds, ds1):
        if d1['start_idx'] != d2['start_idx'] or  d1['end_idx'] != d2['end_idx']:
            print(f"tweet:[{d1['tweet']}] d1!=d2, select: [{d1['select']}]")
            print("d1:%s" % d1['tweet'][d1['offsets'][d1['start_idx']][0]:d1['offsets'][d1['end_idx']][1]])
            print("d2:%s\n" % d2['tweet'][d2['offsets'][d2['start_idx']][0]:d2['offsets'][d2['end_idx']][1]])
            diff_count += 1
        else:
            same_count += 1
    
    print(f"diff:{diff_count}, same:{same_count}")
    
        

In [ ]:
#print(ds1[0])

In [ ]:
import re

if EXPLORE:
    train_df_explore = train_df_explore.fillna(" ")
    train_df_explore[train_df_explore['text'].isna()]

    print(train_df_explore.shape)

    train_df_explore[train_df_explore['text'].str.contains("  ")].shape

    half_word_ids = []

    PRINT_CUT_OFF = 300

    for aa, dd in zip(train_df_explore.iterrows(),ds) :
        index, row = aa
    
        # first word in selected_text:
        space_index = row['selected_text'].find(" ")
        first_word = ""
        if space_index == -1:
            first_word = row['selected_text']
        else:
            first_word = row['selected_text'][:space_index]

        if index < 4:
            print(f"selected_text:{row['selected_text']} first word:{first_word}")
        if len(first_word) == 0:
            continue

        # last word
        space_index = row['selected_text'].rfind(" ")
        if space_index == -1:
            last_word = row['selected_text']
        else:
            last_word = row['selected_text'][space_index+1:]

        if index < 4:
            print(f"selected_text:{row['selected_text']} last word:{last_word}")

        half_word = False
        #find match in original text:
        id_start = 0

        not_found = False

        while id_start < len(row['text']):
            id0 = row['text'].find(first_word, id_start)
            if id0 == -1:
                # not found continue
                not_found = True
                break

            if id0 == 0 or not row['text'][id0-1].isalpha():
                half_word = False
                break
            else:
                half_word = True

            id_start = id0+len(row['text'])

        if not_found:
            continue

        if half_word:
            if index < PRINT_CUT_OFF:
                print(f"\n\nindex:{index} tweet-[{row['text']}]")
                print(f"\n\nindex:{index} selected-[{row['selected_text']}]")
                print(f"\n\nindex:{index} half first word found in [{row['text']}], word:[{first_word}]")
                print(f"\n\nindex:{index} pp-[{prp(row.selected_text, row.text)}]")
                print(dd['tweet'][dd['offsets'][dd['start_idx']+4][0]:dd['offsets'][dd['end_idx']+4][1]])
                

            half_word_ids.append(index)

        # for last word:
        last_half_word = False
        #find match in original text:
        id_start = 0

        while id_start < len(row['text']):
            id0 = row['text'].find(last_word, id_start)
            if index == 50:
                '''
                tokenizer = tokenizers.ByteLevelBPETokenizer(
                    vocab_file='../input/roberta-base/vocab.json', 
                    merges_file='../input/roberta-base/merges.txt', 
                    lowercase=True,
                    add_prefix_space=True)

                tweet = " " + " ".join(row.text.lower().split())
                encoding = tokenizer.encode(tweet)
                print(f"encoding:{encoding}")
                print(f"encoding:{tweet}")
                print(f"encoding:{encoding.ids}")
                print(f"encoding:{encoding.tokens}")
                print(f"encoding:{encoding.offsets}")

                print(f"list: {[ord(i) for i in list(row.text[34:41])]}")
                print(f"list: {[ord(i) for i in list(row.text[-7:])]}")


                print(f"\n{index}-id0:{id0}, found:{row['text'][id0:id0+len(last_word)]}; char:{row['text'][id0+len(last_word)]}, isalpha:{row['text'][id0+len(last_word)].isalpha()}\n")
                '''
                pass

            if id0 == -1:
                break
            if id0+len(last_word)< len(row['text']) and not row['text'][id0+len(last_word)].isalpha():
                last_half_word = False
                break
            elif id0+len(last_word) == len(row['text']):
                last_half_word = False
                break
            else:
                last_half_word = True

            id_start = id0+len(last_word)

        if last_half_word and not half_word:
            if index < PRINT_CUT_OFF:
                print(f"\n\nindex:{index} tweet-[{row['text']}]")
                print(f"\n\nindex:{index} selected-[{row['selected_text']}]")
                print(f"\n\nindex:{index} half last word found in [{row['text']}], word:[{last_word}]")
                print(f"\n\nindex:{index} pp-[{prp(row.selected_text, row.text)}]")
                print(dd['tweet'][dd['offsets'][dd['start_idx']+4][0]:dd['offsets'][dd['end_idx']+4][0]])
                
            half_word_ids.append(index)
        



# Inference

In [ ]:
#%%time

test_df = pd.read_csv(ROOT_PATH + '/input/tweet-sentiment-extraction/test.csv')
test_df['text'] = test_df['text'].astype(str)
test_loader = get_test_loader(test_df)
predictions = []
fold_predictions = []
fold_prediction_scores = []

for i in range(FOLD_COUNT):
  fold_predictions.append([])
  fold_prediction_scores.append([])

models = []
for fold in range(skf.n_splits):
    
    if DEBUG_FOLD:
        if fold != 0:
            print(f"DEBUG infer skip fold:{fold}")
            continue
            
    model = TweetModel()
    model.cuda()
    model.load_state_dict(torch.load(f'roberta_fold{fold+1}.pth'))
    model.eval()
    models.append(model)
    
same_count = 0
diff_count = 0

print("loading data...")
    
fold_2_app_count = 0
for data in test_loader:
    ids = data['ids'].cuda()
    masks = data['masks'].cuda()
    tweet = data['tweet']
    original_tweet = data['original_tweet']
    offsets = data['offsets'].numpy()

    start_logits_list = []
    end_logits_list = []

    #print(f"{start_logits_list}")
    #print(f"processing {data}")
    for model in models:
        with torch.no_grad():
            output = model(ids, masks)
            start_logits_list.append(torch.softmax(output[0], dim=1).cpu().detach().numpy())
            end_logits_list.append(torch.softmax(output[1], dim=1).cpu().detach().numpy())

    start_logits = np.mean(start_logits_list, axis=0)
    end_logits = np.mean(end_logits_list, axis=0)

    
    for i in range(len(ids)):    
        start_pred = np.argmax(start_logits[i])
        end_pred = np.argmax(end_logits[i])
        
        #print(f"processing {i}")
        if start_pred > end_pred:
            pred = tweet[i]
        else:
            pred = get_selected_text(tweet[i], start_pred, end_pred, offsets[i])

            try:
                fifth_pred = postprocess(original_tweet[i], offsets[i], start_pred, end_pred, data['sentiment'])
            except:
                fifth_pred = get_selected_text(tweet[i], start_pred, end_pred, offsets[i])
                
            
            if pred!=fifth_pred:
                diff_count += 1
                #print(f"tweet:[{tweet[i]}], [{pred}]/[{fifth_pred}]")
            else:
                same_count += 1

            pp_pred = pp(pred, original_tweet[i])
            
            pred = fifth_pred
        predictions.append(pred)


    # generate psuedo label lists from models:

    #print(f"len start {len(start_logits_list)}")


    for fold, start_logits, end_logits in zip(range(len(start_logits_list)), start_logits_list, end_logits_list):
      #print(f"fold: {fold} start_logits:{len(start_logits)} end_logits:{len(end_logits)}")
      for i in range(len(ids)):    
          start_pred = np.argmax(start_logits[i])
          start_pred_socre = np.amax(start_logits[i])
          end_pred = np.argmax(end_logits[i])
          end_pred_score = np.amax(end_logits[i])
          
          #print(f"processing {i}")
          if start_pred > end_pred:
              pred = tweet[i]
          else:
              pred = get_selected_text(tweet[i], start_pred, end_pred, offsets[i])

              try:
                  fifth_pred = postprocess(original_tweet[i], offsets[i], start_pred, end_pred, data['sentiment'])
              except:
                  fifth_pred = get_selected_text(tweet[i], start_pred, end_pred, offsets[i])
                  
              
              if pred!=fifth_pred:
                  diff_count += 1
                  #print(f"tweet:[{tweet[i]}], [{pred}]/[{fifth_pred}]")
              else:
                  same_count += 1

              pp_pred = pp(pred, original_tweet[i])
              
              pred = fifth_pred
          fold_predictions[fold].append(pred)
          fold_prediction_scores[fold].append((start_pred_socre + end_pred_score)/2.)

print(f"same:{same_count} diff:{diff_count}")

print(f"fold_2_app_count:{fold_2_app_count}")

# Gen psuedo label files:

In [ ]:
if GEN_PSUEDO_LABELS:
  ori_sub_df = pd.read_csv(ROOT_PATH + '/input/tweet-sentiment-extraction/test.csv')

  for fold, predictions, scores in zip(range(len(fold_predictions)), fold_predictions, fold_prediction_scores):
    sub_df = ori_sub_df.copy(True)

    print(sub_df)
    print(len(fold_predictions[0]))
    print(len(fold_predictions[1]))
    print(len(fold_predictions[2]))
    print(len(fold_predictions[3]))
    print(len(fold_predictions[4]))
    sub_df['selected_text'] = predictions
    sub_df['score'] = scores
    #sub_df['selected_text'] = sub_df['selected_text'].apply(lambda x: x.replace('!!!!', '!') if len(x.split())==1 else x)
    #sub_df['selected_text'] = sub_df['selected_text'].apply(lambda x: x.replace('..', '.') if len(x.split())==1 else x)
    #sub_df['selected_text'] = sub_df['selected_text'].apply(lambda x: x.replace('...', '.') if len(x.split())==1 else x)
    sub_df = sub_df[sub_df.score > .35]

    del sub_df['score']
    sub_df.to_csv("psuedo_labels_fold_%d.csv" % fold,  index=False)
    sub_df.head()

# Submission

In [ ]:
sub_df = pd.read_csv(ROOT_PATH + '/input/tweet-sentiment-extraction/sample_submission.csv')
sub_df['selected_text'] = predictions
#sub_df['selected_text'] = sub_df['selected_text'].apply(lambda x: x.replace('!!!!', '!') if len(x.split())==1 else x)
#sub_df['selected_text'] = sub_df['selected_text'].apply(lambda x: x.replace('..', '.') if len(x.split())==1 else x)
#sub_df['selected_text'] = sub_df['selected_text'].apply(lambda x: x.replace('...', '.') if len(x.split())==1 else x)
sub_df.to_csv('submission.csv', index=False)
sub_df.head()